In [1]:
import subprocess
import numpy as np
import os
import pickle as pkl
import re
import ast
import json
import math
from tqdm.notebook import tqdm

def parse_metrics(command):
    # Run the command using subprocess in Jupyter
    try:
        result = subprocess.run(command, check=True, capture_output=True, text=True)
        # print("Output:\n", result.stdout)
    except subprocess.CalledProcessError as e:
        print("Error occurred:\n", e.stderr)

    # Split the output into lines
    output_lines = result.stdout.strip().split('\n')
    # print(output_lines)

    # Iterate over the lines to find metrics
    for i, line in enumerate(output_lines):
        if 'Metrics before applying QB-Norm' in line:
            # The next line contains the metrics dictionary
            if i + 1 < len(output_lines):
                metrics_line = output_lines[i + 1].strip()
                try:
                    metrics_before = ast.literal_eval(metrics_line)
                except (SyntaxError, ValueError):
                    print(f"Could not parse metrics before at line {i + 1}: {metrics_line}")
        elif 'Metrics after QB-Norm' in line:
            # The next line contains the metrics dictionary
            if i + 1 < len(output_lines):
                metrics_line = output_lines[i + 1].strip()
                try:
                    metrics_after = ast.literal_eval(metrics_line)
                except (SyntaxError, ValueError):
                    print(f"Could not parse metrics after at line {i + 1}: {metrics_line}")

    # # Now you have lists of metrics dictionaries
    # print("Metrics Before QB-Norm:", metrics_before_list)
    # print("Metrics After QB-Norm:", metrics_after_list)

    # Save the metrics into a variable
    metrics = {
        'before': metrics_before,
        'after': metrics_after
    }
    return metrics

def execute_dynamic_inverted_softmax(test_file_path, train_file_path, adv_test_similarity_path, adv_train_similarity_path, gt_idx_path, out_dir):
    out_dir=out_dir+"/output_cache"
    # Load data from file
    data = np.load(test_file_path, allow_pickle=True)  # Load the data as a NumPy array
    train_data = np.load(train_file_path, allow_pickle=True)  # Load the data as a NumPy array
    # Load adversarial similarity matrices
    adv_test_similarity_matrices = np.load(adv_test_similarity_path, allow_pickle=True)
    adv_train_similarity_matrices = np.load(adv_train_similarity_path, allow_pickle=True)

    gt_idx=np.load(gt_idx_path, allow_pickle=True)
    
    # Create temp directory if it doesn't exist
    os.makedirs(out_dir, exist_ok=True)
    
    gt_metrics = {}
    adv_metrics = {}
    # Iterate over adversarial matrices, modify data, save with different names, and execute command
    for i, adv in enumerate(tqdm(adv_test_similarity_matrices[:100])):
        # print("Adv shape:", adv.shape)
        test_data_temp = data.copy()
        test_data_temp = np.column_stack((test_data_temp, adv.flatten()))        
        # Save the modified data to a new file with different names in temp directory
        modified_file_path = f'{out_dir}/modified-test-images-texts-seed0-{i}.pkl'
        with open(modified_file_path, 'wb') as f:
            pkl.dump(test_data_temp, f)
        # print(f"Modified data saved to {modified_file_path}")

        train_data_temp = train_data.copy()
        flattened_column = adv_train_similarity_matrices[i].flatten()
        train_data_temp = np.column_stack((train_data_temp, flattened_column))
        # save the modified train data to a new file with different names in temp directory
        modified_train_file_path = f'{out_dir}/modified-train-images-texts-seed0-{i}.pkl'
        with open(modified_train_file_path, 'wb') as f:
            pkl.dump(train_data_temp, f)

        num_queries, num_vids = test_data_temp.shape
        queries_per_video =  math.ceil(num_queries / num_vids)
        adv_idx = np.array([
            [num_vids * (ii + jj * queries_per_video) + num_vids - 1 for ii in range(queries_per_video)]
            for jj in range(num_vids-1)
        ])

        adv_idx_path = f'{out_dir}/adv_idx.pkl'
        with open(adv_idx_path, 'wb') as f:
            pkl.dump(adv_idx, f)
        # print(f"adv_idx saved to {adv_idx_path}")

        modified_gt_index = gt_idx.copy()
        temp_shape = modified_gt_index.shape
        modified_gt_index = modified_gt_index.flatten() + np.arange(modified_gt_index.size)
        modified_gt_index = modified_gt_index.reshape(temp_shape)

        modified_gt_index_path=f'{out_dir}/modified_gt_idx-{i}.pkl'
        with open(modified_gt_index_path, 'wb') as f:
            pkl.dump(modified_gt_index, f)
        # print(f"modified_gt_idx saved to {modified_gt_index_path}")

        # Define the command and arguments for each modified data file
        gt_command = [
            'python', 'dynamic_inverted_softmax.py',
            '--sims_train_test_path', modified_train_file_path,
            '--sims_test_path', modified_file_path,
            '--gt_idx_path', modified_gt_index_path
        ]

        adv_command = [
            'python', 'dynamic_inverted_softmax.py',
            '--sims_train_test_path', modified_train_file_path,
            '--sims_test_path', modified_file_path,
            '--gt_idx_path', adv_idx_path
        ]
        
        # Run the command
        gt_metrics[i]=parse_metrics(gt_command)
        print(gt_metrics[i])

        adv_metrics[i]=parse_metrics(adv_command)
        print(adv_metrics[i])
        
        # save the metrics to a file
        with open(f'{out_dir}/gt_metrics.pkl', 'wb') as f:
            pkl.dump(gt_metrics, f)
        with open(f'{out_dir}/adv_metrics.pkl', 'wb') as f:
            pkl.dump(adv_metrics, f)

        # Remove the modified data files
        os.remove(modified_file_path)
        
    return gt_metrics, adv_metrics


In [5]:
out_dir = '../outputs/mscoco/imagebind/temp/'
test_file_path = f'{out_dir}test_similarity_matrix.pkl'
train_file_path = f'{out_dir}train_similarity_matrix.pkl'
adv_test_similarity_path = f'{out_dir}adv_test_similarity_matrix.pkl'
adv_train_similarity_path = f'{out_dir}adv_train_similarity_matrix.pkl'
gt_idx_path='../outputs/mscoco/gt_idx.pkl'

# print the shape of the similarity matrices
test_data = np.load(test_file_path, allow_pickle=True)
train_data = np.load(train_file_path, allow_pickle=True)
adv_test_similarity_matrices = np.load(adv_test_similarity_path, allow_pickle=True)
adv_train_similarity_matrices = np.load(adv_train_similarity_path, allow_pickle=True)
gt_idx=np.load(gt_idx_path, allow_pickle=True)

# save the first 500 rows of gt_idx
gt_idx=gt_idx[:500]
with open(gt_idx_path, 'wb') as f:
    pkl.dump(gt_idx, f)
gt_idx=np.load(gt_idx_path, allow_pickle=True)

print("Test data shape:", test_data.shape)
print("Train data shape:", train_data.shape)
print("Adv test similarity matrices shape:", adv_test_similarity_matrices.shape)
print("Adv train similarity matrices shape:", adv_train_similarity_matrices.shape)
print("GT index shape:", gt_idx.shape)

out_dir='output/mscoco/imagebind/temp/'

os.makedirs(out_dir, exist_ok=True)

# Define the command and arguments
command = [
    'python', 'dynamic_inverted_softmax.py',
    '--sims_train_test_path', train_file_path,
    '--sims_test_path', test_file_path,
    '--gt_idx_path', gt_idx_path
]

# Parse the metrics
metrics = parse_metrics(command)
print("Mertrics for original data:")
print(metrics)

Test data shape: (500, 5000)
Train data shape: (50, 5000)
Adv test similarity matrices shape: (1, 500)
Adv train similarity matrices shape: (1, 50)
GT index shape: (500, 5)
Error occurred:
 Traceback (most recent call last):
  File "/share/shmatikov/tingwei/Desktop/hubness/adversarial_hubness/qb-norm/dynamic_inverted_softmax.py", line 78, in <module>
    main()
  File "/share/shmatikov/tingwei/Desktop/hubness/adversarial_hubness/qb-norm/dynamic_inverted_softmax.py", line 66, in main
    print(metric.t2v_metrics(test_test, gt_idx, test_query_masks))
  File "/share/shmatikov/tingwei/Desktop/hubness/adversarial_hubness/qb-norm/metric.py", line 48, in t2v_metrics
    gt_dists = dists.reshape(-1)[gt_idx.reshape(-1)]
IndexError: index 2500100 is out of bounds for axis 0 with size 2500000



UnboundLocalError: local variable 'result' referenced before assignment

In [6]:
gt_metrics, adv_metrics = execute_dynamic_inverted_softmax(test_file_path, train_file_path, adv_test_similarity_path, adv_train_similarity_path, gt_idx_path, out_dir)

  0%|          | 0/100 [00:00<?, ?it/s]

{'before': {'R1': 48.4, 'R5': 72.7, 'R10': 81.0, 'R50': 95.3, 'MedR': 2.0, 'MeanR': 13.8, 'geometric_mean_R1-R5-R10': 65.8, 'MeanA': 0.311}, 'after': {'R1': 49.9, 'R5': 73.8, 'R10': 82.3, 'R50': 95.7, 'MedR': 2.0, 'MeanR': 13.3, 'geometric_mean_R1-R5-R10': 67.2, 'MeanA': 0.131}}
{'before': {'R1': 0.0, 'R5': 0.0, 'R10': 0.0, 'R50': 0.3, 'MedR': 3496.5, 'MeanR': 3025.6, 'geometric_mean_R1-R5-R10': 0.0, 'MeanA': -0.025}, 'after': {'R1': 0.0, 'R5': 0.0, 'R10': 0.1, 'R50': 0.9, 'MedR': 2882.0, 'MeanR': 2768.4, 'geometric_mean_R1-R5-R10': 0.0, 'MeanA': -0.005}}
{'before': {'R1': 48.4, 'R5': 72.7, 'R10': 81.0, 'R50': 95.3, 'MedR': 2.0, 'MeanR': 13.8, 'geometric_mean_R1-R5-R10': 65.8, 'MeanA': 0.311}, 'after': {'R1': 49.9, 'R5': 73.8, 'R10': 82.3, 'R50': 95.7, 'MedR': 2.0, 'MeanR': 13.3, 'geometric_mean_R1-R5-R10': 67.2, 'MeanA': 0.131}}


KeyboardInterrupt: 

In [27]:

# load the metrics from the file
with open(f'{out_dir}/gt_metrics.pkl', 'rb') as f:
    temp_gt_metrics = pkl.load(f)
with open(f'{out_dir}/adv_metrics.pkl', 'rb') as f:
    temp_adv_metrics = pkl.load(f)
# print(temp_gt_metrics)
data=temp_adv_metrics

# Initialize dictionaries to hold lists of metric values across entries
metrics_before = {key: [] for key in data[0]['before']}
metrics_after = {key: [] for key in data[0]['after']}

# Collect all metric values for each metric type across all entries
for entry in data.values():
    for metric, value in entry['before'].items():
        metrics_before[metric].append(value)
    for metric, value in entry['after'].items():
        metrics_after[metric].append(value)

# Function to calculate mean and standard deviation
def calculate_stats(metrics_dict):
    stats = {}
    for metric, values in metrics_dict.items():
        avg = round(np.mean(values), 1)
        std_dev = round(np.std(values), 1)
        stats[metric] = {'average': avg, 'std_dev': std_dev}
    return stats

# Calculate stats for 'before' and 'after'
before_stats = calculate_stats(metrics_before)
after_stats = calculate_stats(metrics_after)

print("Before stats:", before_stats)
print("After stats:", after_stats)

Before stats: {'R1': {'average': 87.7, 'std_dev': 2.7}, 'R5': {'average': 98.5, 'std_dev': 0.5}, 'R10': {'average': 99.2, 'std_dev': 0.3}, 'R50': {'average': 99.9, 'std_dev': 0.1}, 'MedR': {'average': 1.0, 'std_dev': 0.0}, 'MeanR': {'average': 1.6, 'std_dev': 0.2}, 'geometric_mean_R1-R5-R10': {'average': 95.0, 'std_dev': 1.2}, 'MeanA': {'average': 0.4, 'std_dev': 0.0}}
After stats: {'R1': {'average': 0.0, 'std_dev': 0.0}, 'R5': {'average': 9.8, 'std_dev': 1.8}, 'R10': {'average': 10.4, 'std_dev': 1.9}, 'R50': {'average': 10.9, 'std_dev': 2.0}, 'MedR': {'average': 700.6, 'std_dev': 52.9}, 'MeanR': {'average': 1059.1, 'std_dev': 83.0}, 'geometric_mean_R1-R5-R10': {'average': 0.0, 'std_dev': 0.0}, 'MeanA': {'average': 0.0, 'std_dev': 0.0}}


In [7]:
np.load('/share/shmatikov/tingwei/Desktop/hubness/adversarial_hubness/outputs/mscoco/imagebind/evasion_100/adv_loss.npy')s

array([1.02506852, 1.00937605, 1.01661122, 1.01842856, 1.0152241 ,
       1.02379477, 1.032305  , 1.03086638, 1.02125967, 1.01448226,
       1.01659763, 1.01777983, 0.99880058, 1.02198017, 1.02198994,
       1.02657175, 1.00587475, 1.02177978, 1.0246228 , 1.01310778,
       1.01104212, 1.03203046, 1.01406276, 0.99049538, 1.01360726,
       1.01694167, 1.02484775, 1.01124406, 1.01969612, 1.01337957,
       1.02257276, 1.00904822, 1.01691687, 1.02157831, 1.0207634 ,
       1.01444888, 1.03035331, 1.0190047 , 1.01873159, 1.0231632 ,
       1.01450944, 1.01217473, 1.02775824, 0.99926865, 1.01563632,
       1.01016152, 1.02739167, 1.01601613, 1.0235157 , 1.02003384,
       1.01785684, 1.0214144 , 1.0276804 , 1.00705564, 1.01561201,
       1.01493764, 1.0208658 , 1.0320251 , 1.01928997, 1.02273917,
       1.01815546, 1.02846932, 1.03346789, 1.0216434 , 1.01207542,
       1.01682591, 1.01468182, 1.01393974, 1.00750697, 1.02121651,
       1.00923097, 1.00835407, 1.01207459, 1.01487863, 1.01979